# ETL DEL DATASET PRINCIPAL: MOVIES

In [25]:
#Es necesario importar las dependencias necesarias
import pandas as pd
import numpy as np
import json

In [26]:
# Importamos el dataset de trabajo
df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')

# Se elimina las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage']) 
# df.head()
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45466 filas. 


C:\Users\Area-Desa15\AppData\Local\Temp\ipykernel_14620\1522230964.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

In [28]:
# Se accede a la columna "belongs_to_collection" y se visualiza los primeros 10 valores
df["belongs_to_collection"].head(10) 

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
5                                                  NaN
6                                                  NaN
7                                                  NaN
8                                                  NaN
9    {'id': 645, 'name': 'James Bond Collection', '...
Name: belongs_to_collection, dtype: object

In [29]:
# se corrige los campos float
df['belongs_to_collection'].replace('0,065736',None, inplace = True)
df['belongs_to_collection'].replace('1,931659',None, inplace = True)
df['belongs_to_collection'].replace('2,185485',None, inplace = True)

# Se reemplaza el formato de comillas en la columna 'belongs_to_collection'
df['belongs_to_collection'].replace("'",'"', inplace = True) 
# se pasa a string
df['belongs_to_collection'] = df['belongs_to_collection'].astype(str)
subdicc = dict()

for i in range(len(df)):
    franquicia = df.iloc[i]['belongs_to_collection']
    if pd.notna(franquicia):  # Se verifica si no es un valor nulo o NaN
    
        try:
            subdicc = eval(franquicia)  # Usar eval para convertir la cadena en un diccionario
            if isinstance(subdicc, dict):
                #print(subdicc.get('name'))
                df.at[i, 'franquicia'] = subdicc.get('name')
            else:
                #print("El valor no es un diccionario válido.")
                df.at[i, 'franquicia'] = None
        except:
            df.at[i, 'franquicia'] = None
    else:
        df.at[i, 'franquicia'] = None

df['franquicia']

0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: franquicia, Length: 45466, dtype: object

In [30]:
print(df)

                                   belongs_to_collection    budget  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                    nan  65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3                                                    nan  16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...         0   
...                                                  ...       ...   
45461                                                nan         0   
45462                                                nan         0   
45463                                                nan         0   
45464                                                nan         0   
45465                                                nan         0   

                                                  genres      id  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...     862   
1      [{'id': 12, 'nam

In [31]:
lista_productoras = []

for i in range(len(df)):
    lista_productoras = df.iloc[i]['production_companies']

    if pd.notna(lista_productoras):
        # se verifica que lista_productoras no sea un valor nulo

        if isinstance(lista_productoras, str):
            # Si es una cadena, intentar cargarla como JSON
            try:
                lista_productoras = json.loads(lista_productoras.replace("'", "\""))
            except json.JSONDecodeError:
                lista_productoras = None
            # Se pasa la cadena JSON a una lista de diccionarios

        if isinstance(lista_productoras, list):
            # Si es una lista, procesar los diccionarios
            list_productora = []
            for valor in lista_productoras:
                if isinstance(valor, dict):
                    # Si ya es un diccionario, obtener directamente el nombre
                    productora_nombre = valor.get('name')
                    if productora_nombre:
                        list_productora.append(productora_nombre)
                elif isinstance(valor, str):
                    try:
                        # Si es una cadena JSON, cargarla como diccionario
                        diccionario = json.loads(valor.replace("'", "\""))
                        productora_nombre = diccionario.get('name')
                        if productora_nombre:
                            list_productora.append(productora_nombre)
                    except json.JSONDecodeError:
                        pass

            df.at[i, 'productoras'] = list_productora
        else:
            df.at[i, 'productoras'] = None
    else:
        df.at[i, 'productoras'] = None


In [32]:
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia,productoras
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Pixar Animation Studios
1,nan,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,"[TriStar Pictures, Teitler Film, Interscope Co..."
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Warner Bros., Lancaster Gate]"
3,nan,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,[Twentieth Century Fox Film Corporation]
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,"[Sandollar Productions, Touchstone Pictures]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,nan,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,fa,Rising and falling between a man and woman.,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,4.0,1.0,None,[]
45462,nan,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,None,[Sine Olivia]
45463,nan,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0,None,[American World Pictures]
45464,nan,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0,None,[Yermoliev]


In [33]:
# Función que elimine los corchetes '[' y ']' de cada elemento de la lista
def remover_corchetes(lst):
    if isinstance(lst, list):
        return ', '.join(lst)  # Se une los elementos de la lista con una coma y espacio
    return lst

df['productoras'] = df['productoras'].apply(remover_corchetes)


In [11]:
df['productoras']

0                                  Pixar Animation Studios
1        TriStar Pictures, Teitler Film, Interscope Com...
2                             Warner Bros., Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4               Sandollar Productions, Touchstone Pictures
                               ...                        
45461                                                     
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                     
Name: productoras, Length: 45466, dtype: object

In [34]:
lista_paises = []

for i in range(len(df)):
    lista_paises = df.iloc[i]['production_countries']

    if pd.notna(lista_paises):
        # se verifica que la lista de paises no sea un valor nulo

        if isinstance(lista_paises, str):
            # Si es una cadena, intentar cargarla como JSON
            try:
                lista_paises = json.loads(lista_paises.replace("'", "\""))
            except json.JSONDecodeError:
                lista_paises = None
            # Se pasa la cadena JSON a una lista de diccionarios

        if isinstance(lista_paises, list):
            # Si es una lista, procesar los diccionarios
            paises = []
            for valor in lista_paises:
                if isinstance(valor, dict):
                    # Si ya es un diccionario, obtener directamente el nombre
                    nombre = valor.get('name')
                    if nombre:
                        paises.append(nombre)
                elif isinstance(valor, str):
                    try:
                        # Si es una cadena JSON, cargarla como diccionario
                        diccionario = json.loads(valor.replace("'", "\""))
                        nombre = diccionario.get('name')
                        if nombre:
                            paises.append(nombre)
                    except json.JSONDecodeError:
                        pass

            df.at[i, 'paises'] = paises
        else:
            df.at[i, 'paises'] = None
    else:
        df.at[i, 'paises'] = None


In [36]:

df['paises'] = df['paises'].apply(remover_corchetes) # se reutiliza la funcion anterior
df['paises']

0        United States of America
1        United States of America
2        United States of America
3        United States of America
4        United States of America
                   ...           
45461                        Iran
45462                 Philippines
45463    United States of America
45464                      Russia
45465              United Kingdom
Name: paises, Length: 45466, dtype: object

In [37]:
# Se averigua los nulos, con enfoque en los campos revenue y budget .
df.isnull().sum()

belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
franquicia               40975
productoras                490
paises                      11
dtype: int64

In [38]:
# COLUMNA revenue solo tiene nulos, la de budget no.
# Imputamos con el valor 0 los campos nulos de la con una revenue - recaudacion de la pelicula

df['revenue'].fillna(0, inplace=True)

df.isnull().sum()

belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
franquicia               40975
productoras                490
paises                      11
dtype: int64

In [39]:
# Los valores nulos del campo release date (fecha de estreno) deben eliminarse. 
df.dropna(subset='release_date',inplace=True )
# Adicionalmente se hallaron valores 1,12 y 22 en release_date que fueron remplazados por None,
# para luego pasar esta columna a datetime.
df['release_date'].replace('1',None, inplace = True)
df['release_date'].replace('12',None, inplace = True)
df['release_date'].replace('22',None, inplace = True)

df.isnull().sum()


belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 3
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
franquicia               40891
productoras                487
paises                       8
dtype: int64

In [40]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, 
# además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
df['release_date']          = pd.to_datetime(df['release_date'])
df['release_year']          = df['release_date'].dt.year 

df.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia,productoras,paises,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Pixar Animation Studios,United States of America,1995.0
1,nan,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,1995.0
3,nan,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,Twentieth Century Fox Film Corporation,United States of America,1995.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,1995.0


In [41]:
# budge es el presupuesto de la pelicula remplazar los valores incorrectos por cero y luego pasat a float
df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace = True)
df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace = True)
df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace = True)

In [42]:
""" Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
    dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, 
    deberá tomar el valor 0.
"""
# Se convierte los campos 'revenue' y 'budget' a valores numéricos (float)
df['revenue'] = df['revenue'].astype(float)
df['budget'] = df['budget'].astype(float)

# Se calcula el retorno de inversión y reemplaza los valores faltantes con 0
df['retorno'] = np.where((df['revenue'] > 0) & (df['budget'] > 0), df['revenue'] / df['budget'], 0)

df['retorno'] 


0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: retorno, Length: 45379, dtype: float64

In [47]:
df.to_csv('./dataset/movies_limpio.csv', index=False)


PermissionError: [Errno 13] Permission denied: './dataset/movies_limpio.csv'